In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, place_accept, close_trading
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
print(f"Creator address: {storeAppID}")
#close_trading(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: 1906
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)
# storeApp.create_app()
# storeAppID = storeApp.app_id

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App ID: 1906
Store App Address: K54FEXSDDO6SK5LSR46PFGOMALD5JA6TXMZT3DS5RU5ZXVM6JY2PFLYNB4


In [7]:

appID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction RFPZJ65WHLVGK4VWXYDOUVSHVVDGLZHDDIAFWHACCOWRHVA2CVLA confirmed in round 106998.
Waiting for confirmation...
Transaction ODYHFRI5RV2GIN77QM4MSHTXBQJ5JGSTKS64HYKXDRAJDFFDEI7Q confirmed in round 107000.
App ID: 2156
App Address: BXKJQG6JX4OIPRHL7S2O7PVRRKM2TT7PEDBLJOFZYGDBQKFDADG4ML27HU


Alice is setting up and funding token auction...

In [8]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]
print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction KHZ2KOMVZ6U2YFDWA35ZAT5LAPZLSMQBZCPADR3OEJ3GAIW3KGUQ confirmed in round 107002.
Alice's algo balance:  20052208288652  algos


In [9]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 335500, 499: 0}


In [10]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10023975331955  algos
Carla is placing bid for:  1000000  algos


In [11]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 1906
new address: 6VRMOIJJ3RAJLA3FZO2HIU4KLCXTIPA55JR4XQK7VV5CS2SEZYE44ZOYG4
new private_key: DdO7mqeTsCHaPFXxHaZ7FEA6h9GscSNqNaokkNkS+bv1YschKdxAlYNly7R0U4pYrzQ8Hepjy8FfrXopakTOCQ==
new passphrase: giant urban snap denial gentle mask traffic festival sail observe differ access innocent gift grief immune dutch stick fence across rebel dwarf thank absent clean
Waiting for confirmation...
Transaction VMREMODXSWYIQWQIIGM6CQ4GTWQQHIZKLKYLCMY2Y2VYJ6LDB42A confirmed in round 107005.
Waiting for confirmation...
Transaction JZL7LXR6I3SBWIO7VC4NRAMTEBIXOYSJXK4JXFILG46SOG36R7XQ confirmed in round 107007.
Waiting for confirmation...
Transaction RD2UEVQI7ZOUQABDFCHLXXUQ5227XNU6T7DJVKAITAHKPHN7VQ6Q confirmed in round 107009.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1}, 'X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU': {'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1, 'OZ

In [12]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_trade(client, appID, seller, trading_index)
print("Carla cancel trading", trading_index)

Waiting for confirmation...
Transaction EYLHDO7HMU4F7KXHFOEAPRSO3ZVWR77C3R6WL34GCUWA56KT4BUQ confirmed in round 107013.
Carla cancel trading 6VRMOIJJ3RAJLA3FZO2HIU4KLCXTIPA55JR4XQK7VV5CS2SEZYE44ZOYG4


In [13]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=15, price=bidPrice, trading_index=trading_index)

store_app_id 1906
new address: 7H2374LYBHFGWKZP2753RGRPEH7RIECON3QVQUAQYL64YWB4GWR2EBJR2A
new private_key: 1PozJzI2Us8+od2iqeyJLskNlqA4nRJVDCIrSdzI64j59b/xeAnKaysv1/u4mi8h/xQQTm7hWFAQwv3MWDw1ow==
new passphrase: fold soup bean glimpse piece record antique tank crowd razor eager napkin breeze cereal shallow outdoor pelican course much naive review vendor elder above volume
Waiting for confirmation...
Transaction 7H7G3TQGL5YYEBJR7YX5BDJDI6RHDY3YQTG5KRWQSXA5PYCSPXQA confirmed in round 107015.
Waiting for confirmation...
Transaction I4PG63MF2ECSUHMK25UPRNHSMNXRLNVRF54XZLSLPDRGIS5DIABQ confirmed in round 107017.
Waiting for confirmation...
Transaction 5XPXJW2DGNSUIAPPXJL4FKFABNRDKCYGDNIJYRPKVRDF42IWO7EA confirmed in round 107019.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1}, 'X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU': {'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU': 1, 'OZ

'7H2374LYBHFGWKZP2753RGRPEH7RIECON3QVQUAQYL64YWB4GWR2EBJR2A'

In [14]:
place_accept(client, creator, appID, bidder, seller.get_address(), trading_index)

token_amount 15
price 1000000
Waiting for confirmation...
Transaction E6A7QU57FRJORD7MKHXDTLJICA75A2NMORFCR65OJ77NLDZAPAOA confirmed in round 107025.


True

Alice is closing out the auction....

In [15]:
sleep(5)

close_trading(client, appID, creator, [asset_id])

b'assets' [499]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction ALRPGMFJMLGYGRLZTEVVNGDGO667ALWWTHS5FD4KVIUEW3NEWBKA confirmed in round 107028.


In [16]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 335500, 499: 0}


In [17]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 105  for token ID:  499


AssertionError: 

In [19]:
app_global_state = get_app_global_state(client, storeAppID)
print(app_global_state)
seller_app_local_state = get_app_local_state(client, storeAppID, seller.get_address())
print(seller_app_local_state)
bidder_app_local_state = get_app_local_state(client, storeAppID, bidder.get_address())
print(bidder_app_local_state)
# print(encoding.encode_address(app_global_state[b"AS"]))
# print(encoding.encode_address(app_global_state[b"S"]))
#print(encoding.encode_address(storeAppState[b"C"]))

{b'TBA': 0, b'TSA': 0}
{b'SA': 10000000}
{b'BA': 10000000}
